## Bot Telegram

In [1]:
#pip install pyTelegramBotAPI

In [2]:
pip install pyTelegramBotAPI

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Importación de Paquetes

In [3]:



import telebot
from dotenv import load_dotenv
import os
import requests
import yfinance

In [4]:
from telebot.types import InlineKeyboardMarkup
from telebot.types import InlineKeyboardButton

In [5]:
from acciones import create_stocks_markup, get_stock_price, ARGENTINIAN_STOCKS, COMPANY_LOGOS


In [6]:
### Instanciamos el bot de telegram
### el único parámetro obligatorio para instanciarlo es el TOKEN.

In [7]:
# importamos nuestro token
load_dotenv()
TELEGRAM_TOKEN = os.getenv('TELEGRAM_TOKEN')
bot_databanda = telebot.TeleBot(TELEGRAM_TOKEN)

## Responder a menasjes que son comandos (básicos)

In [8]:
# RESPONDER A MENSAJES QUE SON COMANDOS

@bot_databanda.message_handler(commands=['start'])
def cmd_start(message):
    bot_databanda.reply_to(message, "Hola, soy un bot de finanzas. ¿Cómo puedo ayudarte?")

@bot_databanda.message_handler(commands=['fin'])
def cmd_fin(message):
    bot_databanda.reply_to(message, "Nos vemos, recuerda que puedes consultar tu saldo o realizar transferencias.")

@bot_databanda.message_handler(commands=['help'])
def send_help(message):
    bot_databanda.reply_to(message, "Aquí tienes una lista de comandos que puedes usar:\n"
                                     "/start - Iniciar el bot\n"
                                     "/help - Mostrar esta ayuda\n"
                                     "/balance - Consultar tu saldo\n"
                                     "/ingresar - Ingresar monto\n"
                                     "/gasto - Registrar un gasto\n"
                                     "/cotizaciones - Consultar cotizaciones\n")
                                     

## Funciones más complejas (heavys)

In [9]:
#que el usuario pueda consultar su saldo si tiene 0 que pueda agregar en base a transferencia

# Diccionario para guardar el saldo de cada usuario
user_saldos = {}
from telebot import types  # Asegúrate de tener esto arriba


@bot_databanda.message_handler(commands=['gasto'])
def cmd_gasto(message):
    bot_databanda.reply_to(message, "💸 Ingresa el monto:")
    bot_databanda.register_next_step_handler(message, process_gasto)

def process_gasto(message):
    try:
        amount = int(message.text)
        if amount <= 0:
            msg = bot_databanda.reply_to(
                message,
                "⚠️ El monto debe ser mayor que 0. Por favor, ingresa un monto válido:"
            )
            bot_databanda.register_next_step_handler(msg, process_gasto)
            return
        
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
        markup.add("Sí", "No")
        msg = bot_databanda.send_message(
            message.chat.id,
            f"Has ingresado un monto de ${amount}. ¿Deseas proceder con el ingreso?",
            reply_markup=markup
        )
        bot_databanda.register_next_step_handler(msg, confirm_gasto, amount)

    except ValueError:
        msg = bot_databanda.reply_to(
            message,
            "❌ Ingresa solo números (ejemplo: 1500). Por favor, intenta de nuevo:"
        )
        bot_databanda.register_next_step_handler(msg, process_gasto)

def confirm_gasto(message, amount):
    respuesta = message.text.strip().lower()
    user_id = message.from_user.id
    if respuesta in ["si", "sí"]:
        # Sumar el monto al saldo del usuario
        user_saldos[user_id] = user_saldos.get(user_id, 1000) - amount
        bot_databanda.send_message(message.chat.id, f"✅ Gasto de ${amount} registrado con éxito.")
    elif respuesta == "no":
        bot_databanda.send_message(message.chat.id, "❌ Gasto cancelado.")
    else:
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
        markup.add("Sí", "No")
        msg = bot_databanda.send_message(
            message.chat.id,
            "Por favor, responde usando los botones: ¿Deseas proceder con el gasto?",
            reply_markup=markup
        )
        bot_databanda.register_next_step_handler(msg, confirm_gasto, amount)

@bot_databanda.message_handler(commands=['ingresar'])
def cmd_ingresar(message):
    bot_databanda.reply_to(message, "💸 Ingresa el monto:")
    bot_databanda.register_next_step_handler(message, process_amount)

def process_amount(message):
    try:
        amount = int(message.text)
        if amount <= 0:
            msg = bot_databanda.reply_to(
                message,
                "⚠️ El monto debe ser mayor que 0. Por favor, ingresa un monto válido:"
            )
            bot_databanda.register_next_step_handler(msg, process_amount)
            return
        
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
        markup.add("Sí", "No")
        msg = bot_databanda.send_message(
            message.chat.id,
            f"Has ingresado un monto de ${amount}. ¿Deseas proceder con el ingreso?",
            reply_markup=markup
        )
        bot_databanda.register_next_step_handler(msg, confirm_ingreso, amount)

    except ValueError:
        msg = bot_databanda.reply_to(
            message,
            "❌ Ingresa solo números (ejemplo: 1500). Por favor, intenta de nuevo:"
        )
        bot_databanda.register_next_step_handler(msg, process_amount)

def confirm_ingreso(message, amount):
    respuesta = message.text.strip().lower()
    user_id = message.from_user.id
    if respuesta in ["si", "sí"]:
        # Sumar el monto al saldo del usuario
        user_saldos[user_id] = user_saldos.get(user_id, 1000) + amount
        bot_databanda.send_message(message.chat.id, f"✅ Ingreso de ${amount} realizada con éxito.")
    elif respuesta == "no":
        bot_databanda.send_message(message.chat.id, "❌ Ingreso cancelado.")
    else:
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
        markup.add("Sí", "No")
        msg = bot_databanda.send_message(
            message.chat.id,
            "Por favor, responde usando los botones: ¿Deseas proceder con el ingreso?",
            reply_markup=markup
        )
        bot_databanda.register_next_step_handler(msg, confirm_ingreso, amount)

@bot_databanda.message_handler(commands=['balance'])
def send_balance(message):
    user_id = message.from_user.id
    saldo = user_saldos.get(user_id, 1000)  # Saldo inicial por defecto
    bot_databanda.reply_to(message, f"Tu saldo es de ${saldo}")

In [10]:
# RESPONDER A MENSAJES QUE SON TEXTO
#@bot_databanda.message_handler(content_types=['text'])

def bot_mensajes_testo(message):
    bot_databanda.reply_to(message.chat.id, "Me enviaste un mensaje de texto: " + message)


In [11]:
# seteand comandos
bot_databanda.set_my_commands([
    telebot.types.BotCommand("/start", "Iniciar el bot"),
    telebot.types.BotCommand("/help", "Mostrar ayuda"),
    telebot.types.BotCommand("/balance", "Consultar tu saldo"),
    telebot.types.BotCommand("/ingresar", "Realizar un ingreso"),
    telebot.types.BotCommand("/gasto", "Registrar un gasto"),
    telebot.types.BotCommand("/cotizacion", "Consultar cotizaciones"),
    telebot.types.BotCommand("/acciones", "Consultar acciones"),
    telebot.types.BotCommand("/fin", "Finalizar la conversación")
])

True

## OBTENER COTIZACIÓN DÓLAR

In [12]:
@bot_databanda.message_handler(commands=['cotizaciones'])
def cmd_cotizaciones(message):
    markup = InlineKeyboardMarkup()
    # Botones para cada moneda
    botones = [
        InlineKeyboardButton("💵 Dólar Oficial", callback_data="oficial"),
        InlineKeyboardButton("🔵 Dólar Blue", callback_data="blue"),
        InlineKeyboardButton("📈 Dólar MEP", callback_data="mep"),
        InlineKeyboardButton("💶 Euro", callback_data="euro"),
        InlineKeyboardButton("🇧🇷 Real", callback_data="real")
    ]
    for b in botones:
        markup.add(b)
    bot_databanda.send_message(message.chat.id, "Selecciona la cotización que quieras ver:", reply_markup=markup)

# Función para obtener cotización desde la API
import requests

def get_cotizacion(tipo):
    url = "https://dolarapi.com/v1/dolares"
    try:
        r = requests.get(url)
        if r.status_code == 200:
            data = r.json()
            # Buscar la casa de cambio que coincide con el tipo
            casa = next((item for item in data if item['casa'].lower() == tipo.lower()), None)
            if casa:
                return f"{casa['nombre']} ({casa['moneda']}):\nCompra: ${casa['compra']} - Venta: ${casa['venta']}\nÚltima actualización: {casa['fechaActualizacion']}"
            else:
                return f"❌ No se encontró la cotización para '{tipo}'."
        else:
            return "❌ Error al obtener los datos de la API."
    except Exception as e:
        return f"❌ Ocurrió un error: {e}"


@bot_databanda.callback_query_handler(func=lambda call: True)
def callback_cotizacion(call):
    if call.data.startswith("stock_"):
        stock_name = call.data.split("stock_")[1]

        # Caso "Todas las acciones"
        if stock_name == "all":
            message_text = "📊 *Cotizaciones de todas las acciones:*\n\n"
            for name, ticker_symbol in ARGENTINIAN_STOCKS.items():
                message_text += get_stock_price(ticker_symbol) + "\n"

            bot_databanda.send_message(
                chat_id=call.message.chat.id,
                text=message_text,
                parse_mode='Markdown'
            )
        else:
            ticker_symbol = ARGENTINIAN_STOCKS.get(stock_name)
            if ticker_symbol:
                message_text = get_stock_price(ticker_symbol)
                logo_url = COMPANY_LOGOS.get(stock_name)

                if logo_url:
                    with open(logo_url, "rb") as photo:
                        bot_databanda.send_photo(
                            chat_id=call.message.chat.id,
                            photo=photo,
                            caption=message_text,
                            parse_mode='Markdown'
                        )
                else:
                    bot_databanda.send_message(
                        chat_id=call.message.chat.id,
                        text=message_text,
                        parse_mode='Markdown'
                    )
    else:
        cotizacion = get_cotizacion(call.data)
        bot_databanda.edit_message_text(chat_id=call.message.chat.id,
                                        message_id=call.message.message_id,
                                        text=cotizacion)


# Nuevo comando para mostrar el menú de acciones
@bot_databanda.message_handler(commands=['acciones'])
def cmd_acciones(message):
    markup = create_stocks_markup()
    bot_databanda.send_message(message.chat.id, "Selecciona la acción que quieras ver:", reply_markup=markup)



## Lanzar el programa /bot

In [ ]:
if __name__ == "__main__":
    print("Iniciando el bot...")

    # Seteamos los comandos de nuevo ...
    bot_databanda.set_my_commands([
        telebot.types.BotCommand("/start", "Iniciar el bot"),
        telebot.types.BotCommand("/help", "Mostrar ayuda"),
        telebot.types.BotCommand("/balance", "Consultar tu saldo"),
        telebot.types.BotCommand("/ingresar", "Realizar un ingreso"),
        telebot.types.BotCommand("/gasto", "Registrar un gasto"),
        telebot.types.BotCommand("/cotizaciones", "Consultar cotizaciones"),
        telebot.types.BotCommand("/acciones", "Consultar acciones"),
        telebot.types.BotCommand("/fin", "Finalizar la conversación"),
        
    ])

    bot_databanda.infinity_polling() # bucle infinito para recibir mensajes (while true)

    print("Fin = Bot Detenido")


    

Iniciando el bot...
